housing price prediction.

TODO.
* Replace null value to specific values. See [missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html), [One hot](https://www.kaggle.com/alexisbcook/categorical-variables).
* Other Model like keras.

DONE.
* Version 3: Drop missing columns.(train loss 180XX -> submit: 19817).
* Version 7: Fill missing number columns with SimpleImputer.
  * NaN values in number columns are filled with 'mean(default)'. (loss 175XX.)
  * NaN(np.nan) values in object columns are filled with 'most_frequent'. (Train loss 170XX -> Submit: 20514)
* Version 8: Add Keras Model. (Submit: 46590)
* Version 9~ : Optimize Keras.

In [1]:
import glob

import pandas as pd

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [2]:
print(f'total col num: {len(train.columns)}')
print(f'object column names: {train.select_dtypes(include="object").columns.tolist()}')
print(f'number column names: {train.select_dtypes(include="number").columns.tolist()}')
print(f'Null exist columns in train: {train.columns[train.isna().any()].tolist()}')
print(f'Null exist columns in test: {test.columns[test.isna().any()].tolist()}')

total col num: 81
object column names: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
number column names: ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt'

In [3]:
test_ids = test.Id
null_cols_train = train.columns[train.isna().any()].tolist()
null_cols_test = test.columns[test.isna().any()].tolist()

In [4]:
sorted(train[null_cols_train].nunique().to_dict().items(), key=lambda x: x[1], reverse=True)[:5]

[('MasVnrArea', 327),
 ('LotFrontage', 110),
 ('GarageYrBlt', 97),
 ('BsmtFinType1', 6),
 ('BsmtFinType2', 6)]

In [5]:
sorted(test[null_cols_test].nunique().to_dict().items(), key=lambda x: x[1], reverse=True)[:5]

[('BsmtUnfSF', 793),
 ('TotalBsmtSF', 736),
 ('BsmtFinSF1', 669),
 ('GarageArea', 459),
 ('MasVnrArea', 303)]

In [6]:
from sklearn.impute import SimpleImputer

number_columns = train.select_dtypes(include='number').columns.tolist()
number_columns.remove('SalePrice')
my_imputer = SimpleImputer(strategy='mean')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(train[number_columns]))
imputed_X_test = pd.DataFrame(my_imputer.fit_transform(test[number_columns]))

imputed_X_train.columns = train[number_columns].columns
imputed_X_test.columns = test[number_columns].columns

In [7]:
for number_column in number_columns:
    train[number_column] = imputed_X_train[number_column]
    test[number_column] = imputed_X_test[number_column]

In [8]:
import numpy as np

object_columns = train.select_dtypes(include='object').columns.tolist()
most_freq_imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputed_X_train = pd.DataFrame(most_freq_imputer.fit_transform(train[object_columns]))
imputed_X_test = pd.DataFrame(most_freq_imputer.fit_transform(test[object_columns]))

imputed_X_train.columns = train[object_columns].columns
imputed_X_test.columns = test[object_columns].columns

for object_column in object_columns:
    train[object_column] = imputed_X_train[object_column]
    test[object_column] = imputed_X_test[object_column]

In [9]:
test_ids = test.Id
null_cols_train = train.columns[train.isna().any()].tolist()
null_cols_test = test.columns[test.isna().any()].tolist()
null_cols = set(null_cols_train) | set(null_cols_test)
null_cols.add('Id')
print(set(number_columns) & set(null_cols))
print(set(object_columns) & set(null_cols))

{'Id'}
set()


In [10]:
train = train.drop(columns = null_cols)
test = test.drop(columns = null_cols)

In [11]:
set(train.columns.tolist()) ^ set(test.columns.tolist())

{'SalePrice'}

In [12]:
from sklearn.preprocessing import LabelEncoder

object_cols = train.select_dtypes(include='object').columns
label_encoder = LabelEncoder()
for col in object_cols:
    print(col)
    train[col] = label_encoder.fit_transform(train[col])   
    test[col] = label_encoder.fit_transform(test[col])

MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


In [13]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
for col in train.columns:
    if col == 'SalePrice':
        continue
    col_val = train[col].values
    col_val_scaled = min_max_scaler.fit_transform(col_val.reshape(-1,1))
    train[col] = col_val_scaled
    
    col_val = test[col].values
    col_val_scaled = min_max_scaler.fit_transform(col_val.reshape(-1,1))
    test[col] = col_val_scaled

In [14]:
from sklearn.model_selection import train_test_split

y = train.SalePrice
X = train.drop(['SalePrice'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def RandomForest(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    print(mean_absolute_error(y_valid, preds))
    return model

In [16]:
random_forest = RandomForest(X_train, X_valid, y_train, y_valid)

17068.869897260272


In [17]:
import collections
forest_dict = collections.defaultdict(float)

for idx, test_id in enumerate(test_ids):
     forest_dict[test_id] = random_forest.predict([test.iloc[idx].values])

new_forest_dict = {}
new_forest_dict['Id'] = [int(Id) for Id in forest_dict.keys()]
new_forest_dict['SalePrice'] = [ float(value) for value in forest_dict.values()]

In [18]:
result = pd.DataFrame.from_dict(new_forest_dict)
result
result.to_csv('/kaggle/working/forest_submission.csv', index=False)

In [19]:
import tensorflow as tf

def build_keras_model(X_train):
    keras_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=[len(X_train.columns.tolist())]),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
        ])
    
    optimizer = tf.keras.optimizers.Adam(0.01)
    keras_model.compile(loss='mse',
                        optimizer=optimizer,
                        metrics=['mae', 'mse'])
    return keras_model


In [20]:
keras_model = build_keras_model(X_train)

In [21]:
keras_model.predict(X_train[:10])

array([[-0.0314002 ],
       [-0.28758287],
       [-0.45704234],
       [-0.2554793 ],
       [-0.31706524],
       [-0.24188928],
       [-0.02387852],
       [-0.20192948],
       [-0.00124363],
       [-0.30687225]], dtype=float32)

In [22]:
history = keras_model.fit(X_train,
                    y_train,
                    validation_data=(X_valid, y_valid),
                    epochs=100,
                    verbose=1)

Epoch 1/100
37/37 [==============================] - 0s 7ms/step - loss: 38761500672.0000 - mae: 180569.1094 - mse: 38761500672.0000 - val_loss: 39378198528.0000 - val_mae: 180210.1094 - val_mse: 39378198528.0000
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 36684238848.0000 - mae: 174773.6875 - mse: 36684238848.0000 - val_loss: 34218342400.0000 - val_mae: 165404.4375 - val_mse: 34218342400.0000
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 26428092416.0000 - mae: 141889.3906 - mse: 26428092416.0000 - val_loss: 18261686272.0000 - val_mae: 107504.5391 - val_mse: 18261686272.0000
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 10179689472.0000 - mae: 72633.3203 - mse: 10179689472.0000 - val_loss: 6581886464.0000 - val_mae: 52770.9297 - val_mse: 6581886464.0000
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 5861586432.0000 - mae: 56454.7773 - mse: 5861586432.0000 - val_loss: 6194306560

In [23]:
keras_results = keras_model.predict(test)

keras_test_dict = collections.defaultdict(float)
for test_id, result in zip(test_ids, keras_results):
    keras_test_dict[int(test_id)] = float(result)

In [24]:
new_keras_dict = {}
new_keras_dict['Id'] = [int(Id) for Id in keras_test_dict.keys()]
new_keras_dict['SalePrice'] = [ float(value) for value in keras_test_dict.values()]

result = pd.DataFrame.from_dict(new_keras_dict)
print(result)
result.to_csv('/kaggle/working/keras_submission.csv', index=False)

        Id      SalePrice
0     1461  100762.562500
1     1462  139884.859375
2     1463  165249.812500
3     1464  173413.281250
4     1465  154607.140625
...    ...            ...
1454  2915   59240.746094
1455  2916   58079.554688
1456  2917  165077.765625
1457  2918  106334.398438
1458  2919  212524.593750

[1459 rows x 2 columns]
